In [ ]:
# CELL 1: SIMPLE PACKAGE INSTALLER FOR LIGHTNING.AI
import os
import sys
import subprocess
import re
from typing import List, Dict, Any

print("🔍 Setting up simple Lightning.ai package installer...")

class SimpleLightningInstaller:
    """Simple package installer for Lightning.ai - no bullshit."""
    
    def __init__(self):
        # Basic conda-to-pip package mappings
        self.package_mappings = {
            # PyTorch packages
            'pytorch': 'torch',
            'pytorch-cuda': 'torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118',
            'pytorch-cuda=11.8': 'torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118',
            'pytorch-cuda=11.7': 'torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117',
            'torchvision': 'torchvision',
            'torchaudio': 'torchaudio',
            
            # TensorFlow packages
            'tensorflow': 'tensorflow',
            'tensorflow-gpu': 'tensorflow',
            
            # ML packages
            'scikit-learn': 'scikit-learn',
            'sklearn': 'scikit-learn',
            'opencv': 'opencv-python',
            'opencv-python': 'opencv-python',
            
            # Data science
            'pandas': 'pandas',
            'numpy': 'numpy',
            'matplotlib': 'matplotlib',
            'seaborn': 'seaborn',
            'plotly': 'plotly',
            
            # Jupyter
            'jupyter': 'jupyter',
            'notebook': 'notebook',
            'ipywidgets': 'ipywidgets',
            
            # Hugging Face
            'transformers': 'transformers',
            'datasets': 'datasets',
            'accelerate': 'accelerate',
            'tokenizers': 'tokenizers',
            
            # Utilities
            'tqdm': 'tqdm',
            'click': 'click',
            'rich': 'rich',
            'typer': 'typer',
            'requests': 'requests',
            'urllib3': 'urllib3',
            
            # Testing
            'pytest': 'pytest',
            'unittest': 'unittest',
            
            # Web
            'flask': 'flask',
            'fastapi': 'fastapi',
            'uvicorn': 'uvicorn',
            'starlette': 'starlette',
            
            # Database
            'sqlalchemy': 'sqlalchemy',
            'psycopg2': 'psycopg2-binary',
            'pymysql': 'pymysql',
            
            # Other
            'pillow': 'pillow',
            'Pillow': 'pillow',
            'beautifulsoup4': 'beautifulsoup4',
            'bs4': 'beautifulsoup4',
            'lxml': 'lxml',
            'yaml': 'pyyaml',
            'pyyaml': 'pyyaml',
        }
        
        # Packages to skip (system dependencies)
        self.skip_packages = [
            'cudatoolkit',
            'cudnn',
            'cuda',
            'mkl',
            'intel',
            'gcc',
            'build-essential',
            'cmake',
            'make',
        ]
    
    def detect_lightning(self) -> bool:
        """Detect if we're running on Lightning.ai."""
        # Check environment variables
        lightning_vars = ['LIGHTNING_TEAM_ID', 'LIGHTNING_PROJECT_ID', 'LIGHTNING_STUDIO_ID']
        for var in lightning_vars:
            if var in os.environ:
                return True
        
        # Check paths
        lightning_paths = ['/teamspace', '/studio', '/shared']
        for path in lightning_paths:
            if os.path.exists(path):
                return True
        
        return False
    
    def convert_package(self, conda_pkg: str) -> str:
        """Convert a conda package to pip format."""
        # Extract base package name (remove version specs)
        base_pkg = re.split(r'[>=<==]', conda_pkg)[0].strip()
        
        # Check if it should be skipped
        if base_pkg in self.skip_packages:
            return f"# SKIPPED (system dependency): {conda_pkg}"
        
        # Check mappings
        if base_pkg in self.package_mappings:
            pip_pkg = self.package_mappings[base_pkg]
            # Preserve version specifications if any
            version_spec = re.sub(r'^[a-zA-Z0-9_-]+', '', conda_pkg)
            if version_spec and '=' not in pip_pkg and not pip_pkg.startswith('#'):
                return f"{pip_pkg}{version_spec}"
            return pip_pkg
        
        # If no mapping, try to use as-is (most packages have same name)
        if not any(char in conda_pkg for char in ['=', '>', '<']):
            return conda_pkg
        
        return f"# NO MAPPING: {conda_pkg}"
    
    def convert_packages(self, conda_packages: List[str]) -> Dict[str, Any]:
        """Convert a list of conda packages to pip format."""
        result = {
            'success': True,
            'pip_packages': [],
            'skipped_packages': [],
            'failed_packages': [],
            'warnings': [],
        }
        
        for pkg in conda_packages:
            converted = self.convert_package(pkg)
            
            if converted.startswith('# SKIPPED'):
                result['skipped_packages'].append(pkg)
            elif converted.startswith('# NO MAPPING'):
                result['failed_packages'].append(pkg)
                result['warnings'].append(f"No mapping for: {pkg}")
            else:
                result['pip_packages'].append(converted)
        
        return result
    
    def install_packages(self, packages: List[str]) -> Dict[str, Any]:
        """Install packages using pip."""
        result = {
            'success': False,
            'installed_packages': [],
            'failed_packages': [],
            'errors': [],
            'execution_time': 0,
        }
        
        if not packages:
            result['success'] = True
            return result
        
        import time
        start_time = time.time()
        
        try:
            for pkg in packages:
                print(f"📦 Installing {pkg}...")
                
                # Skip comments
                if pkg.strip().startswith('#'):
                    print(f"   ⏭️  Skipping: {pkg}")
                    continue
                
                # Install using pip
                cmd = [sys.executable, '-m', 'pip', 'install', pkg]
                
                try:
                    process = subprocess.run(
                        cmd,
                        capture_output=True,
                        text=True,
                        timeout=300  # 5 minutes timeout
                    )
                    
                    if process.returncode == 0:
                        result['installed_packages'].append(pkg)
                        print(f"   ✅ Success: {pkg}")
                    else:
                        result['failed_packages'].append(pkg)
                        error_msg = process.stderr.strip() or process.stdout.strip()
                        result['errors'].append(f"{pkg}: {error_msg}")
                        print(f"   ❌ Failed: {pkg}")
                        
                except subprocess.TimeoutExpired:
                    result['failed_packages'].append(pkg)
                    result['errors'].append(f"Timeout installing: {pkg}")
                    print(f"   ⏰ Timeout: {pkg}")
                    
                except Exception as e:
                    result['failed_packages'].append(pkg)
                    result['errors'].append(f"Exception installing {pkg}: {str(e)}")
                    print(f"   💥 Exception: {pkg}")
            
            result['success'] = len(result['failed_packages']) == 0
            
        except Exception as e:
            result['errors'].append(f"Installation failed: {str(e)}")
        
        result['execution_time'] = time.time() - start_time
        return result
    
    def install_conda_packages(self, conda_packages: List[str]) -> Dict[str, Any]:
        """Convert and install conda packages."""
        print(f"🔄 Converting {len(conda_packages)} conda packages to pip...")
        
        # Convert packages
        conversion_result = self.convert_packages(conda_packages)
        
        print(f"✅ Conversion complete:")
        print(f"   Pip packages: {len(conversion_result['pip_packages'])}")
        print(f"   Skipped: {len(conversion_result['skipped_packages'])}")
        print(f"   Failed: {len(conversion_result['failed_packages'])}")
        
        if conversion_result['warnings']:
            print(f"⚠️  Warnings:")
            for warning in conversion_result['warnings']:
                print(f"   • {warning}")
        
        # Install converted packages
        if conversion_result['pip_packages']:
            print(f"\n📦 Installing {len(conversion_result['pip_packages'])} pip packages...")
            install_result = self.install_packages(conversion_result['pip_packages'])
            
            # Combine results
            final_result = {
                'success': install_result['success'],
                'converted_packages': conversion_result['pip_packages'],
                'skipped_packages': conversion_result['skipped_packages'],
                'failed_conversions': conversion_result['failed_packages'],
                'installed_packages': install_result['installed_packages'],
                'failed_packages': install_result['failed_packages'],
                'errors': conversion_result['warnings'] + install_result['errors'],
                'execution_time': install_result['execution_time'],
            }
            
            return final_result
        else:
            return {
                'success': False,
                'converted_packages': [],
                'skipped_packages': conversion_result['skipped_packages'],
                'failed_conversions': conversion_result['failed_packages'],
                'installed_packages': [],
                'failed_packages': [],
                'errors': conversion_result['warnings'],
                'execution_time': 0,
            }

# Create installer instance
installer = SimpleLightningInstaller()

# Check environment
is_lightning = installer.detect_lightning()
print(f"\n🌍 Environment: {'Lightning.ai' if is_lightning else 'Other'}")

if is_lightning:
    print("✅ Lightning.ai detected - using pip-only installation")
else:
    print("ℹ️  Not Lightning.ai - still using pip-only installation")

print("🔧 Simple installer ready!")

In [ ]:
# CELL 2: INSTALL BASIC PACKAGES
if 'installer' not in locals():
    print("❌ Run Cell 1 first!")
else:
    print("📦 Installing basic packages...")
    
    basic_packages = [
        "numpy",
        "pandas",
        "requests",
        "tqdm",
        "click",
        "rich"
    ]
    
    result = installer.install_packages(basic_packages)
    
    print(f"\n📊 Result:")
    print(f"   Success: {result['success']}")
    print(f"   Installed: {len(result['installed_packages'])}")
    print(f"   Failed: {len(result['failed_packages'])}")
    print(f"   Time: {result['execution_time']:.1f}s")
    
    if result['installed_packages']:
        print(f"\n✅ Installed:")
        for pkg in result['installed_packages']:
            print(f"   • {pkg}")
    
    if result['failed_packages']:
        print(f"\n❌ Failed:")
        for i, pkg in enumerate(result['failed_packages']):
            print(f"   • {pkg}")
            if i < len(result['errors']):
                print(f"     Error: {result['errors'][i]}")

In [ ]:
# CELL 3: CONVERT AND INSTALL CONDA PACKAGES
if 'installer' not in locals():
    print("❌ Run Cell 1 first!")
else:
    print("🔄 Testing conda-to-pip conversion...")
    
    # Test conda packages that would normally fail on Lightning.ai
    conda_packages = [
        "pytorch",                    # Should convert to 'torch'
        "pytorch-cuda=11.8",         # Should convert to pip with CUDA
        "torchvision",                # Should map directly
        "cudatoolkit=11.8",          # Should be skipped
        "transformers",               # Should map directly
        "pandas",                     # Should map directly
        "numpy",                      # Should map directly
        "scikit-learn",               # Should convert to 'scikit-learn'
        "opencv",                     # Should convert to 'opencv-python'
        "matplotlib",                 # Should map directly
    ]
    
    result = installer.install_conda_packages(conda_packages)
    
    print(f"\n📊 Conversion & Installation Result:")
    print(f"   Success: {result['success']}")
    print(f"   Converted: {len(result['converted_packages'])}")
    print(f"   Skipped: {len(result['skipped_packages'])}")
    print(f"   Failed conversions: {len(result['failed_conversions'])}")
    print(f"   Installed: {len(result['installed_packages'])}")
    print(f"   Failed installations: {len(result['failed_packages'])}")
    print(f"   Time: {result['execution_time']:.1f}s")
    
    if result['converted_packages']:
        print(f"\n✅ Converted packages:")
        for pkg in result['converted_packages']:
            print(f"   • {pkg}")
    
    if result['skipped_packages']:
        print(f"\n⏭️  Skipped packages (system deps):")
        for pkg in result['skipped_packages']:
            print(f"   • {pkg}")
    
    if result['failed_conversions']:
        print(f"\n❌ Failed conversions:")
        for pkg in result['failed_conversions']:
            print(f"   • {pkg}")
    
    if result['installed_packages']:
        print(f"\n🎉 Successfully installed:")
        for pkg in result['installed_packages']:
            print(f"   • {pkg}")
    
    if result['failed_packages']:
        print(f"\n💥 Failed to install:")
        for i, pkg in enumerate(result['failed_packages']):
            print(f"   • {pkg}")
            if i < len(result['errors']):
                print(f"     Error: {result['errors'][i]}")

In [ ]:
# CELL 4: CUSTOM PACKAGE INSTALLATION
if 'installer' not in locals():
    print("❌ Run Cell 1 first!")
else:
    print("🎯 Custom Package Installation")
    print("=" * 40)
    
    # Example: Install packages for a specific use case
    print("\n🤖 Installing ML/DL packages...")
    
    ml_packages = [
        "torch",                      # PyTorch
        "torchvision",                # Computer vision
        "transformers",              # NLP
        "datasets",                  # Dataset management
        "accelerate",                # Training acceleration
        "evaluate",                  # Evaluation metrics
        "wandb",                      # Experiment tracking
    ]
    
    result = installer.install_packages(ml_packages)
    
    print(f"\n📊 ML Packages Result:")
    print(f"   Success: {result['success']}")
    print(f"   Installed: {len(result['installed_packages'])}")
    print(f"   Failed: {len(result['failed_packages'])}")
    
    if result['installed_packages']:
        print(f"\n✅ ML packages ready:")
        for pkg in result['installed_packages']:
            print(f"   • {pkg}")
    
    if result['failed_packages']:
        print(f"\n❌ Failed ML packages:")
        for i, pkg in enumerate(result['failed_packages']):
            print(f"   • {pkg}")
            if i < len(result['errors']):
                print(f"     Error: {result['errors'][i]}")

In [ ]:
# CELL 5: REPO CLONING AND INTEGRATION SETUP
if 'installer' not in locals():
    print("❌ Run Cell 1 first!")
else:
    print("🔗 Setting up repository integration...")
    
    try:
        import subprocess
        import os
        
        # Clone the repository
        repo_url = "https://github.com/your-repo/SD-Pinnokio.git"
        clone_dir = "/tmp/SD-Pinnokio"
        
        print(f"📥 Cloning repository to {clone_dir}...")
        
        # Remove existing directory if it exists
        if os.path.exists(clone_dir):
            subprocess.run(["rm", "-rf", clone_dir])
        
        # Clone the repository
        clone_result = subprocess.run(
            ["git", "clone", repo_url, clone_dir],
            capture_output=True,
            text=True
        )
        
        if clone_result.returncode == 0:
            print("✅ Repository cloned successfully!")
            
            # Check what's in the repository
            if os.path.exists(clone_dir):
                print(f"\n📁 Repository contents:")
                for item in os.listdir(clone_dir):
                    item_path = os.path.join(clone_dir, item)
                    if os.path.isdir(item_path):
                        print(f"   📁 {item}/")
                    else:
                        print(f"   📄 {item}")
            
            # Try to import from the repository
            print(f"\n🔍 Testing repository imports...")
            
            # Add the repository to Python path
            if clone_dir not in sys.path:
                sys.path.insert(0, clone_dir)
            
            # Try to import key modules
            test_modules = [
                "notebook_ui",
                "complete_notebook_ui",
                "conda_converter"
            ]
            
            for module in test_modules:
                try:
                    exec(f"import {module}")
                    print(f"   ✅ {module} imported successfully")
                except ImportError as e:
                    print(f"   ❌ {module} import failed: {e}")
            
        else:
            print(f"❌ Clone failed: {clone_result.stderr}")
            
    except Exception as e:
        print(f"❌ Repository setup failed: {e}")

In [ ]:
# CELL 6: TEST INSTALLED PACKAGES (FIXED)
if 'installer' not in locals():
    print("❌ Run Cell 1 first!")
else:
    print("🧪 Testing installed packages...")
    
    test_packages = [
        ("numpy", "import numpy as np; print('NumPy version:', np.__version__)"),
        ("pandas", "import pandas as pd; print('Pandas version:', pd.__version__)"),
        ("torch", "import torch; print('PyTorch version:', torch.__version__)"),
        ("transformers", "import transformers; print('Transformers version:', transformers.__version__)"),
        ("matplotlib", "import matplotlib.pyplot as plt; print('Matplotlib version:', plt.__version__)"),
    ]
    
    success_count = 0
    total_count = len(test_packages)
    
    for pkg_name, test_code in test_packages:
        print(f"\n🔍 Testing {pkg_name}...")
        try:
            exec(test_code)
            print(f"   ✅ {pkg_name} works!")
            success_count += 1
        except ImportError:
            print(f"   ❌ {pkg_name} not installed")
        except Exception as e:
            print(f"   ⚠️  {pkg_name} error: {str(e)}")
    
    print(f"\n📊 Test Results: {success_count}/{total_count} packages working")
    
    if success_count == total_count:
        print("🎉 All packages are working correctly!")
    elif success_count > 0:
        print("✅ Some packages working - environment is usable")
    else:
        print("❌ No packages working - check installation")

In [ ]:
# CELL 7: SUMMARY AND USAGE EXAMPLES (FIXED)
if 'installer' not in locals():
    print("❌ Run Cell 1 first!")
else:
    print("📝 Lightning.ai Package Installer Summary")
    print("=" * 50)
    
    is_lightning = installer.detect_lightning()
    print(f"\n🌍 Platform: {'Lightning.ai' if is_lightning else 'Other'}")
    
    print(f"\n✅ What this installer provides:")
    print(f"   1. Conda-to-pip package conversion")
    print(f"   2. Automatic Lightning.ai detection")
    print(f"   3. Simple pip-only installation")
    print(f"   4. No complex dependencies")
    print(f"   5. Error handling and logging")
    
    print(f"\n🚀 Usage Examples:")
    print(f"\n# Install basic packages")
    print(f"result = installer.install_packages(['numpy', 'pandas', 'torch'])")
    
    print(f"\n# Convert and install conda packages")
    print(f"conda_pkgs = ['pytorch', 'cudatoolkit=11.8', 'transformers']")
    print(f"result = installer.install_conda_packages(conda_pkgs)")
    
    print(f"\n# Convert single package")
    pip_pkg = installer.convert_package('pytorch-cuda=11.8')
    print(f"print('Converted:', pip_pkg)")
    
    print(f"\n💡 Supported Conda Packages:")
    supported_pkgs = list(installer.package_mappings.keys())
    for i, pkg in enumerate(supported_pkgs[:20]):  # Show first 20
        pip_equiv = installer.package_mappings[pkg]
        print(f"   {pkg} → {pip_equiv}")
    
    if len(supported_pkgs) > 20:
        print(f"   ... and {len(supported_pkgs) - 20} more")
    
    print(f"\n⚠️  Skipped Packages (system dependencies):")
    for pkg in installer.skip_packages:
        print(f"   • {pkg}")
    
    print(f"\n🎉 Ready to use! No more conda bullshit on Lightning.ai!")